### O objetvo do projeto é definir um critério de depreciação e encontrar oportunidades de mercado no ramo dos carros.

### Lendo o aqruivo .csv e deixando a coluna "ano_mes" no formato ```date time```.

In [3]:
import pandas as pd
import numpy as np

fipe = pd.read_csv("tabela_fipe_carros.csv", sep = ",", header=0, index_col = False)
fipe["ano_mes"] = pd.to_datetime(fipe["ano_mes"], format="%Y%m")

### Começo do ```Data Frame```.

In [4]:
fipe

,ano_mes,marca,modelo,codigo_ano,ano_modelo,codigo_tipo_combustivel,combustivel,valor,codigo_fipe,codigo_tipo_veiculo
0,2001-01-01,Alfa Romeo,145 Elegant 1.7/1.8 16V,1999-1,1999,1,Gasolina,28961.0,006009-7,1
1,2001-01-01,Alfa Romeo,145 Elegant 1.7/1.8 16V,1998-1,1998,1,Gasolina,26827.0,006009-7,1
2,2001-01-01,Alfa Romeo,145 Elegant 1.7/1.8 16V,1996-1,1996,1,Gasolina,22494.0,006009-7,1
3,2001-01-01,Alfa Romeo,145 Elegant 1.7/1.8 16V,1995-1,1995,1,Gasolina,21509.0,006009-7,1
4,2001-01-01,Alfa Romeo,145 Elegant 2.0 16V,1998-1,1998,1,Gasolina,26764.0,006001-1,1
...,...,...,...,...,...,...,...,...,...,...
5072994,2024-10-01,VW - VolksWagen,VOYAGE Trendline 1.0 T.Flex 12V 4p,2017-1,2017,1,Gasolina,43568.0,005457-7,1
5072995,2024-10-01,VW - VolksWagen,VOYAGE Trendline 1.6 T.Flex 8V 4p,2018-1,2018,1,Gasolina,45992.0,005380-5,1
5072996,2024-10-01,VW - VolksWagen,VOYAGE Trendline 1.6 T.Flex 8V 4p,2017-1,2017,1,Gasolina,43288.0,005380-5,1
5072997,2024-10-01,VW - VolksWagen,VOYAGE Trendline 1.6 T.Flex 8V 4p,2016-1,2016,1,Gasolina,41983.0,005380-5,1


### O critério que eu criei se baseou na interpretação dos valores dos carros como retornos financeiros, ou seja, no primeiro dia registrado seria o quanto você investiu e no segundo o quanto aquela aplicação (no caso o carro) rendeu pra você, os dois itens que eu considerei para aplicar nesse modelo, foram o retorno composto e a volatilidade, ao final do notebook, citarei temas que poderiam ser inseridos ao modelo para melhorá-lo e por consequência, melhorar a tomada dedecisão. Além disso será calculado a depreciação dos carros, para ver quais carros menos perderam valor com o passar do tempo.

### A primeira etapa consistiu entender como o arquivo estva organizado. Ao manipular o ```Data Frame``` percebe-se que ao agrupá-lo por "codigo_fipe" e "ano_modelo", a coluna "ano_mes" fica em sequência, porém, há casos em que há buracos nessas sequências e é isso que a função

<code>
def find_missing_months(group, date_range):
    existing_dates = group['ano_mes'].unique()
    missing_dates = date_range[~date_range.isin(existing_dates)]
    return missing_dates
</code>

### faz. Ela os acha e retorna um ```Data Frame``` com todos eles.

In [6]:
intervalo = pd.date_range(start='2001-01-01', end='2024-10-01', freq='MS')

def find_missing_months(group, date_range):
    existing_dates = group['ano_mes'].unique()
    missing_dates = date_range[~date_range.isin(existing_dates)]
    return missing_dates

missing_months = fipe.groupby(['codigo_fipe', 'codigo_ano']).apply(find_missing_months, date_range=intervalo, include_groups=False)

In [7]:
missing_months

codigo_fipe  codigo_ano
001001-4     1992-1        DatetimeIndex(['2001-01-01', '2001-02-01', '20...
             1993-1        DatetimeIndex([], dtype='datetime64[ns]', freq...
             1994-1        DatetimeIndex([], dtype='datetime64[ns]', freq...
             1995-1        DatetimeIndex([], dtype='datetime64[ns]', freq...
             1995-2        DatetimeIndex([], dtype='datetime64[ns]', freq...
                                                 ...                        
097002-6     32000-1       DatetimeIndex(['2001-01-01', '2001-02-01', '20...
098001-3     2023-1        DatetimeIndex(['2001-01-01', '2001-02-01', '20...
             32000-1       DatetimeIndex(['2001-01-01', '2001-02-01', '20...
098002-1     2023-1        DatetimeIndex(['2001-01-01', '2001-02-01', '20...
             32000-1       DatetimeIndex(['2001-01-01', '2001-02-01', '20...
Length: 34615, dtype: object

### Verificando se todos os códigos fipe estão aqui.

In [8]:
index = set([i[0] for i in missing_months.index])
len(index)

6762

In [9]:
len(fipe["codigo_fipe"].unique())

6762

### É possível notar que a função está funcionando, ao comparar com o ```Data Frame```. Para o código fipe igual a 006009-7 e modelo do ano 1997, faltam valores de todos os meses do ano de 2001 e de 2002 faltam os meses de janeiro até outubro.

In [10]:
missing_months.loc['006009-7'].loc['1997-1']

DatetimeIndex(['2001-01-01', '2001-02-01', '2001-03-01', '2001-04-01',
               '2001-05-01', '2001-06-01', '2001-07-01', '2001-08-01',
               '2001-09-01', '2001-10-01', '2001-11-01', '2001-12-01',
               '2002-01-01', '2002-02-01', '2002-03-01', '2002-04-01',
               '2002-05-01', '2002-06-01', '2002-07-01', '2002-08-01',
               '2002-09-01', '2002-10-01'],
              dtype='datetime64[ns]', freq='MS')

In [11]:
fipe[(fipe["codigo_fipe"] == '006009-7') & (fipe["codigo_ano"] == '1997-1')]

,ano_mes,marca,modelo,codigo_ano,ano_modelo,codigo_tipo_combustivel,combustivel,valor,codigo_fipe,codigo_tipo_veiculo
181677,2002-11-01,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997,1,Gasolina,17864.0,006009-7,1
190967,2002-12-01,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997,1,Gasolina,17769.0,006009-7,1
200368,2003-01-01,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997,1,Gasolina,17863.0,006009-7,1
209844,2003-02-01,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997,1,Gasolina,18113.0,006009-7,1
219359,2003-03-01,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997,1,Gasolina,17643.0,006009-7,1
...,...,...,...,...,...,...,...,...,...,...
4933957,2024-06-01,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997,1,Gasolina,14946.0,006009-7,1
4961619,2024-07-01,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997,1,Gasolina,14898.0,006009-7,1
4989371,2024-08-01,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997,1,Gasolina,14848.0,006009-7,1
5017182,2024-09-01,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997,1,Gasolina,14815.0,006009-7,1


### Após entender a organização do aqruivo, partiu-se para o tratamento dos dados. Como mencionado, há buracos no ```Data Frame```, esses buracos eu os tratei com interpolação linear. A função logo abaixo completava esses buracos. Ela primeiro identifica qual ```range``` de datas do arquivo, cria um ```Data Frame``` com essas datas, e une com o original. Essa união completa, para cada código fipe e modelo, os meses faltantes e a interpolação coloca os valores correspondentes. Porém ao interpolar só é possível saber o valor entre dois valores conhecidos, antes não é possível saber e depois também. Para os valores anteriores a função completa com ```NaN``` e os valores depois permanecem constantes igual o último valor disponível.

```
def add_missing_dates_and_interpolate(group, date_range):
    full_dates = pd.DataFrame(date_range, columns=['ano_mes'])
    merged = pd.merge(full_dates, group, on='ano_mes', how='left')
    merged[['codigo_fipe', 'marca', 'modelo', 'codigo_ano', 'ano_modelo', 'codigo_tipo_combustivel', 'combustivel', 'codigo_tipo_veiculo']] = \
    merged[['codigo_fipe', 'marca', 'modelo', 'codigo_ano', 'ano_modelo', 'codigo_tipo_combustivel', 'combustivel', 'codigo_tipo_veiculo']].ffill().bfill()
    merged['valor'] = merged['valor'].interpolate(method='linear')
    return merged
```


In [17]:
def add_missing_dates_and_interpolate(group, date_range):
    # Criar um DataFrame com todas as datas no intervalo
    full_dates = pd.DataFrame(date_range, columns=['ano_mes'])

    # Resetar o índice para que codigo_fipe e codigo_ano se tornem colunas
    group = group.reset_index()

    # Mesclar com o grupo original para adicionar as datas faltantes
    merged = pd.merge(full_dates, group, on='ano_mes', how='left')

    # Preencher as colunas que não são 'valor' com os valores do grupo
    merged[['codigo_fipe', 'marca', 'modelo', 'ano_modelo', 'codigo_tipo_combustivel', 'combustivel', 'codigo_tipo_veiculo', 'codigo_ano']] = \
        merged[['codigo_fipe', 'marca', 'modelo', 'ano_modelo', 'codigo_tipo_combustivel', 'combustivel', 'codigo_tipo_veiculo', 'codigo_ano']].ffill().bfill()

    # Interpolar os valores da coluna 'valor'
    merged['valor'] = merged['valor'].interpolate(method='linear')

    return merged

# Aplicar a função para cada grupo de codigo_fipe e ano_modelo
interpolado = fipe.groupby(['codigo_fipe', 'codigo_ano'], group_keys=False).apply(
    add_missing_dates_and_interpolate, date_range=intervalo
)

<ipython-input-17-38bb80a5b31c>:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  interpolado = fipe.groupby(['codigo_fipe', 'codigo_ano'], group_keys=False).apply(


### É possível ver que o ```Data Frame``` já está completo, com os meses inde de janeiro de 2001 até outubro de 2024.

In [18]:
interpolado[(interpolado["codigo_fipe"] == '006009-7') & (interpolado["codigo_ano"] == '1997-1')]

,ano_mes,index,marca,modelo,codigo_ano,ano_modelo,codigo_tipo_combustivel,combustivel,valor,codigo_fipe,codigo_tipo_veiculo
0,2001-01-01,NaN,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997.0,1.0,Gasolina,NaN,006009-7,1.0
1,2001-02-01,NaN,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997.0,1.0,Gasolina,NaN,006009-7,1.0
2,2001-03-01,NaN,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997.0,1.0,Gasolina,NaN,006009-7,1.0
3,2001-04-01,NaN,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997.0,1.0,Gasolina,NaN,006009-7,1.0
4,2001-05-01,NaN,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997.0,1.0,Gasolina,NaN,006009-7,1.0
...,...,...,...,...,...,...,...,...,...,...,...
281,2024-06-01,4933957.0,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997.0,1.0,Gasolina,14946.0,006009-7,1.0
282,2024-07-01,4961619.0,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997.0,1.0,Gasolina,14898.0,006009-7,1.0
283,2024-08-01,4989371.0,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997.0,1.0,Gasolina,14848.0,006009-7,1.0
284,2024-09-01,5017182.0,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997-1,1997.0,1.0,Gasolina,14815.0,006009-7,1.0


### Para tratar os valores ```NaN```, eu simplesmente os igualei ao primeiro valor não nulo na coluna valor e os valores constantes que ficam até o final eu mantive, pelo simples motivo de que eles não alteram o retorno, então ao inserir esses valores constantes não há problema de enviezar o modelo.

In [37]:
def replace_nan_with_first_valid(group):
    # Encontrar o primeiro valor não NaN na coluna 'valor'
    first_valid_index = group['valor'].first_valid_index()

    # Se houver um valor válido, substituir os NaN por ele
    if first_valid_index is not None:
        # Usar .loc para acessar o valor e preencher
        fill_value = group.loc[first_valid_index, 'valor']
        group['valor'].fillna(fill_value, inplace=True)

    return group

# Crie uma cópia mutável dos nomes dos níveis do índice
new_index_names = list(interpolado.index.names)
changed_names = False

# Verifique se 'codigo_fipe' é um nome de nível do índice E um nome de coluna
try:
    # .name é para índice único, .names para MultiIndex ou índice único
    idx_names_list = interpolado.index.names if interpolado.index.nlevels > 1 else [interpolado.index.name]

    if 'codigo_fipe' in idx_names_list and 'codigo_fipe' in interpolado.columns:
        idx_pos = idx_names_list.index('codigo_fipe')
        new_index_names[idx_pos] = 'codigo_fipe_idx' # Renomeie o nível do índice
        changed_names = True

    # Faça o mesmo para 'codigo_ano', caso também esteja causando conflito
    if 'codigo_ano' in idx_names_list and 'codigo_ano' in interpolado.columns:
        idx_pos = idx_names_list.index('codigo_ano')
        new_index_names[idx_pos] = 'codigo_ano_idx' # Renomeie o nível do índice
        changed_names = True

    if changed_names:
        interpolado.index.names = new_index_names

except AttributeError:
    # Pode acontecer se o índice não tiver nome(s) (todos None)
    pass

interpolado = interpolado.groupby(['codigo_fipe', 'codigo_ano']).apply(replace_nan_with_first_valid, include_groups=False)

<ipython-input-37-e8f187e878f9>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  group['valor'].fillna(fill_value, inplace=True)


In [42]:
interpolado = interpolado.reset_index()

In [43]:
interpolado[(interpolado["codigo_fipe"] == '006009-7') & (interpolado["codigo_ano"] == '1997-1')]

,codigo_fipe,codigo_ano,codigo_fipe_idx,codigo_ano_idx,level_4,ano_mes,index,marca,modelo,ano_modelo,codigo_tipo_combustivel,combustivel,valor,codigo_tipo_veiculo
3816670,006009-7,1997-1,006009-7,1997-1,0,2001-01-01,NaN,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,17864.0,1.0
3816671,006009-7,1997-1,006009-7,1997-1,1,2001-02-01,NaN,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,17864.0,1.0
3816672,006009-7,1997-1,006009-7,1997-1,2,2001-03-01,NaN,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,17864.0,1.0
3816673,006009-7,1997-1,006009-7,1997-1,3,2001-04-01,NaN,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,17864.0,1.0
3816674,006009-7,1997-1,006009-7,1997-1,4,2001-05-01,NaN,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,17864.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816951,006009-7,1997-1,006009-7,1997-1,281,2024-06-01,4933957.0,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,14946.0,1.0
3816952,006009-7,1997-1,006009-7,1997-1,282,2024-07-01,4961619.0,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,14898.0,1.0
3816953,006009-7,1997-1,006009-7,1997-1,283,2024-08-01,4989371.0,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,14848.0,1.0
3816954,006009-7,1997-1,006009-7,1997-1,284,2024-09-01,5017182.0,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,14815.0,1.0


### Para poder comparar os valores é necessário trazer todos para o mesmo período, basta corrigí-los pela inflação. Para isso foi utilizado uma série histórica do IPCA de 2001 até 2024.

In [52]:
ipca = pd.read_csv("ipca_historico.csv", sep = ",", header=0, index_col = False)
ipca['ano_mes'] = pd.to_datetime(ipca['ano_mes'])

ipca = ipca.dropna()
ipca

,ano_mes,indice,var_pct,fator_acumulado,fator_de_atualizacao_out_2024
0,2001-01-01,1693.07,0.005702507,1.000000,4.155959
1,2001-02-01,1700.86,0.004601109,1.004601,4.136925
2,2001-03-01,1707.32,0.003798079,1.008417,4.121272
3,2001-04-01,1717.22,0.005798561,1.014264,4.097512
4,2001-05-01,1724.26,0.004099649,1.018422,4.080782
...,...,...,...,...,...
281,2024-06-01,6941.51,0.002100489,4.099955,1.013660
282,2024-07-01,6967.89,0.003800326,4.115536,1.009822
283,2024-08-01,6966.50,-0.000199487,4.114715,1.010024
284,2024-09-01,6997.15,0.004399627,4.132818,1.005599


In [53]:
corrigido = interpolado.merge(ipca[['ano_mes', 'fator_de_atualizacao_out_2024']], on='ano_mes', how='left')
corrigido


,codigo_fipe,codigo_ano,codigo_fipe_idx,codigo_ano_idx,level_4,ano_mes,index,marca,modelo,ano_modelo,codigo_tipo_combustivel,combustivel,valor,codigo_tipo_veiculo,fator_de_atualizacao_out_2024
0,001001-4,1992-1,001001-4,1992-1,0,2001-01-01,NaN,Fiat,Fiorino Furgão 1.5 mpi / i.e.,1992.0,1.0,Gasolina,5746.0,1.0,4.155959
1,001001-4,1992-1,001001-4,1992-1,1,2001-02-01,NaN,Fiat,Fiorino Furgão 1.5 mpi / i.e.,1992.0,1.0,Gasolina,5746.0,1.0,4.136925
2,001001-4,1992-1,001001-4,1992-1,2,2001-03-01,NaN,Fiat,Fiorino Furgão 1.5 mpi / i.e.,1992.0,1.0,Gasolina,5746.0,1.0,4.121272
3,001001-4,1992-1,001001-4,1992-1,3,2001-04-01,NaN,Fiat,Fiorino Furgão 1.5 mpi / i.e.,1992.0,1.0,Gasolina,5746.0,1.0,4.097512
4,001001-4,1992-1,001001-4,1992-1,4,2001-05-01,NaN,Fiat,Fiorino Furgão 1.5 mpi / i.e.,1992.0,1.0,Gasolina,5746.0,1.0,4.080782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9899891,098002-1,32000-1,098002-1,32000-1,281,2024-06-01,4956414.0,SERES,SERES 5 BEV (Elétrico),32000.0,1.0,Gasolina,400000.0,1.0,1.013660
9899892,098002-1,32000-1,098002-1,32000-1,282,2024-07-01,4984149.0,SERES,SERES 5 BEV (Elétrico),32000.0,1.0,Gasolina,394990.0,1.0,1.009822
9899893,098002-1,32000-1,098002-1,32000-1,283,2024-08-01,5011963.0,SERES,SERES 5 BEV (Elétrico),32000.0,1.0,Gasolina,394990.0,1.0,1.010024
9899894,098002-1,32000-1,098002-1,32000-1,284,2024-09-01,5039830.0,SERES,SERES 5 BEV (Elétrico),32000.0,1.0,Gasolina,394990.0,1.0,1.005599


In [54]:
corrigido['valor_corrigido'] = corrigido['valor'] * corrigido['fator_de_atualizacao_out_2024']
corrigido

,codigo_fipe,codigo_ano,codigo_fipe_idx,codigo_ano_idx,level_4,ano_mes,index,marca,modelo,ano_modelo,codigo_tipo_combustivel,combustivel,valor,codigo_tipo_veiculo,fator_de_atualizacao_out_2024,valor_corrigido
0,001001-4,1992-1,001001-4,1992-1,0,2001-01-01,NaN,Fiat,Fiorino Furgão 1.5 mpi / i.e.,1992.0,1.0,Gasolina,5746.0,1.0,4.155959,23880.142098
1,001001-4,1992-1,001001-4,1992-1,1,2001-02-01,NaN,Fiat,Fiorino Furgão 1.5 mpi / i.e.,1992.0,1.0,Gasolina,5746.0,1.0,4.136925,23770.770188
2,001001-4,1992-1,001001-4,1992-1,2,2001-03-01,NaN,Fiat,Fiorino Furgão 1.5 mpi / i.e.,1992.0,1.0,Gasolina,5746.0,1.0,4.121272,23680.828539
3,001001-4,1992-1,001001-4,1992-1,3,2001-04-01,NaN,Fiat,Fiorino Furgão 1.5 mpi / i.e.,1992.0,1.0,Gasolina,5746.0,1.0,4.097512,23544.305434
4,001001-4,1992-1,001001-4,1992-1,4,2001-05-01,NaN,Fiat,Fiorino Furgão 1.5 mpi / i.e.,1992.0,1.0,Gasolina,5746.0,1.0,4.080782,23448.176136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9899891,098002-1,32000-1,098002-1,32000-1,281,2024-06-01,4956414.0,SERES,SERES 5 BEV (Elétrico),32000.0,1.0,Gasolina,400000.0,1.0,1.013660,405463.940800
9899892,098002-1,32000-1,098002-1,32000-1,282,2024-07-01,4984149.0,SERES,SERES 5 BEV (Elétrico),32000.0,1.0,Gasolina,394990.0,1.0,1.009822,398869.670383
9899893,098002-1,32000-1,098002-1,32000-1,283,2024-08-01,5011963.0,SERES,SERES 5 BEV (Elétrico),32000.0,1.0,Gasolina,394990.0,1.0,1.010024,398949.255338
9899894,098002-1,32000-1,098002-1,32000-1,284,2024-09-01,5039830.0,SERES,SERES 5 BEV (Elétrico),32000.0,1.0,Gasolina,394990.0,1.0,1.005599,397201.716091


In [55]:
corrigido[(corrigido["codigo_fipe"] == '006009-7') & (corrigido["codigo_ano"] == '1997-1')]

,codigo_fipe,codigo_ano,codigo_fipe_idx,codigo_ano_idx,level_4,ano_mes,index,marca,modelo,ano_modelo,codigo_tipo_combustivel,combustivel,valor,codigo_tipo_veiculo,fator_de_atualizacao_out_2024,valor_corrigido
3816670,006009-7,1997-1,006009-7,1997-1,0,2001-01-01,NaN,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,17864.0,1.0,4.155959,74242.056810
3816671,006009-7,1997-1,006009-7,1997-1,1,2001-02-01,NaN,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,17864.0,1.0,4.136925,73902.025520
3816672,006009-7,1997-1,006009-7,1997-1,2,2001-03-01,NaN,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,17864.0,1.0,4.121272,73622.401847
3816673,006009-7,1997-1,006009-7,1997-1,3,2001-04-01,NaN,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,17864.0,1.0,4.097512,73197.958977
3816674,006009-7,1997-1,006009-7,1997-1,4,2001-05-01,NaN,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,17864.0,1.0,4.080782,72899.098241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816951,006009-7,1997-1,006009-7,1997-1,281,2024-06-01,4933957.0,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,14946.0,1.0,1.013660,15150.160148
3816952,006009-7,1997-1,006009-7,1997-1,282,2024-07-01,4961619.0,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,14898.0,1.0,1.009822,15044.331121
3816953,006009-7,1997-1,006009-7,1997-1,283,2024-08-01,4989371.0,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,14848.0,1.0,1.010024,14996.831675
3816954,006009-7,1997-1,006009-7,1997-1,284,2024-09-01,5017182.0,Alfa Romeo,145 Elegant 1.7/1.8 16V,1997.0,1.0,Gasolina,14815.0,1.0,1.005599,14897.955452


### Os dados já foram tratados então parte-se para os cálculos. A fórumla de retorno a ser utilizada é a seguinte

<code>
(returns + 1).prod() - 1) * 100

### Os retornos são obtidos utilizando a função, ```pct_change()```, disponível na biblioteca do ```Pandas```. Como o exmplo abaixo,o carro de código fipe igual a 090001-0 e modelo do ano de 2018 teve um retorno de -68,3%.

In [56]:
((corrigido[(corrigido["codigo_fipe"] == '090001-0') & (corrigido["codigo_ano"] == '2018-1')]['valor_corrigido'].pct_change() + 1 ).prod() - 1)*100

np.float64(-68.28628329339928)

In [57]:
retornos = corrigido[['codigo_fipe', 'codigo_ano', 'ano_mes', 'valor_corrigido']]
retornos

,codigo_fipe,codigo_ano,ano_mes,valor_corrigido
0,001001-4,1992-1,2001-01-01,23880.142098
1,001001-4,1992-1,2001-02-01,23770.770188
2,001001-4,1992-1,2001-03-01,23680.828539
3,001001-4,1992-1,2001-04-01,23544.305434
4,001001-4,1992-1,2001-05-01,23448.176136
...,...,...,...,...
9899891,098002-1,32000-1,2024-06-01,405463.940800
9899892,098002-1,32000-1,2024-07-01,398869.670383
9899893,098002-1,32000-1,2024-08-01,398949.255338
9899894,098002-1,32000-1,2024-09-01,397201.716091


### Aplicando a todo o ```Data Frame```. E criando uma ```Serie``` com o retorno composto de todos os carros e seus respectivos modelos.

In [58]:
# 1. Calcular a variação percentual por grupo
df_resultado = corrigido.groupby(['codigo_fipe', 'codigo_ano'])['valor_corrigido'].agg(
    lambda x: ((x.pct_change().dropna() + 1).prod() - 1) * 100
).reset_index()

df_resultado.rename(columns={'valor_corrigido': 'retorno_percentual'}, inplace = True)

df_resultado

,codigo_fipe,codigo_ano,retorno_percentual
0,001001-4,1992-1,-62.223843
1,001001-4,1993-1,-55.040187
2,001001-4,1994-1,-59.751245
3,001001-4,1995-1,-63.311439
4,001001-4,1995-2,-71.586125
...,...,...,...
34610,097002-6,32000-1,-75.938167
34611,098001-3,2023-1,-76.489647
34612,098001-3,32000-1,-75.938167
34613,098002-1,2023-1,-76.489296


### Contando em quantos períodos o retorno foi calculado.

In [59]:
periodos = (
    fipe.groupby(['codigo_fipe', 'codigo_ano'])['ano_mes']
    .agg(['min', 'max'])
    .reset_index()
)
periodos

,codigo_fipe,codigo_ano,min,max
0,001001-4,1992-1,2002-04-01,2024-10-01
1,001001-4,1993-1,2001-01-01,2024-10-01
2,001001-4,1994-1,2001-01-01,2024-10-01
3,001001-4,1995-1,2001-01-01,2024-10-01
4,001001-4,1995-2,2001-01-01,2024-10-01
...,...,...,...,...
34610,097002-6,32000-1,2023-05-01,2024-10-01
34611,098001-3,2023-1,2024-01-01,2024-10-01
34612,098001-3,32000-1,2024-01-01,2024-10-01
34613,098002-1,2023-1,2024-01-01,2024-10-01


In [60]:
import datetime

# Calculate the difference in days
delta = (periodos["max"] - periodos["min"]) / np.timedelta64(1, 'D')
delta = (delta/30.44).astype(int)
# Convert the difference in days to months
# Using an average month length of 30.44 days
delta = delta.to_frame(name = "n_meses")

In [61]:
delta

,n_meses
0,270
1,284
2,284
3,284
4,284
...,...
34610,17
34611,9
34612,9
34613,9


### Adicionando os períodos ao ```Data Frame``` dos retornos.

In [62]:
df_resultado["n_meses"] = list(delta["n_meses"])

In [63]:
df_resultado

,codigo_fipe,codigo_ano,retorno_percentual,n_meses
0,001001-4,1992-1,-62.223843,270
1,001001-4,1993-1,-55.040187,284
2,001001-4,1994-1,-59.751245,284
3,001001-4,1995-1,-63.311439,284
4,001001-4,1995-2,-71.586125,284
...,...,...,...,...
34610,097002-6,32000-1,-75.938167,17
34611,098001-3,2023-1,-76.489647,9
34612,098001-3,32000-1,-75.938167,9
34613,098002-1,2023-1,-76.489296,9


### Ordenando os retornos.

In [64]:
df_resultado.sort_values(by = ["retorno_percentual", "n_meses"], ascending = False)

,codigo_fipe,codigo_ano,retorno_percentual,n_meses
7527,004055-0,1988-1,82.794917,284
10437,005027-0,1985-1,82.198269,284
7529,004055-0,1990-1,81.226718,284
7528,004055-0,1989-1,80.405181,284
10443,005027-0,1996-1,79.303877,284
...,...,...,...,...
29852,029059-9,2005-1,-97.253958,236
11515,005184-5,2005-1,-97.436922,236
29812,029050-5,2001-1,-97.456100,281
11514,005184-5,2004-1,-97.597620,250


In [65]:
# Filtrar os modelos com volatilidade abaixo da mediana e retorno acima da mediana
retorno_corte_moderado = df_resultado['retorno_percentual'].median()

df_moderado = df_resultado[(df_resultado['retorno_percentual'] > retorno_corte_moderado)]

# Ordenar por retorno (do maior para o menor)
df_moderado = df_moderado.sort_values(by='retorno_percentual', ascending=False)

In [66]:
df_moderado[(df_moderado['retorno_percentual'] > 0)]

,codigo_fipe,codigo_ano,retorno_percentual,n_meses
7527,004055-0,1988-1,82.794917,284
10437,005027-0,1985-1,82.198269,284
7529,004055-0,1990-1,81.226718,284
7528,004055-0,1989-1,80.405181,284
10443,005027-0,1996-1,79.303877,284
10440,005027-0,1994-1,72.084143,284
10441,005027-0,1995-1,71.936518,284
7520,004054-1,1989-1,68.558614,284
10438,005027-0,1986-1,66.611159,284
7523,004054-1,1992-1,65.934651,284


### Calculando o retorno anual de cada carro, no período

In [72]:
df_moderado['retorno_anual'] = ((1 + df_moderado['retorno_percentual'] / 100) ** (12 / df_moderado['n_meses']) - 1) * 100
df_moderado[(df_moderado['retorno_anual'] > 0)]


,codigo_fipe,codigo_ano,retorno_percentual,n_meses,retorno_anual
7527,004055-0,1988-1,82.794917,284,2.581467
10437,005027-0,1985-1,82.198269,284,2.567297
7529,004055-0,1990-1,81.226718,284,2.544128
7528,004055-0,1989-1,80.405181,284,2.524444
10443,005027-0,1996-1,79.303877,284,2.497921
10440,005027-0,1994-1,72.084143,284,2.320082
10441,005027-0,1995-1,71.936518,284,2.316372
7520,004054-1,1989-1,68.558614,284,2.230627
10438,005027-0,1986-1,66.611159,284,2.180442
7523,004054-1,1992-1,65.934651,284,2.162877


Os carros que mais valorizaram no período foram:

1.   Chevrolet Opala Diplomata/Diplom
2.   Volkswagen Fusca 1600
3.   Chevrolet Opala Comodoro/Comod


In [70]:
df_depreciacao = corrigido.groupby(['codigo_fipe', 'codigo_ano'])['valor_corrigido'].apply(
    lambda x: (x.pct_change().dropna())
).reset_index()

df_depreciacao.rename(columns={'valor_corrigido': 'depreciacao_mensal'}, inplace = True)
df_depreciacao

,codigo_fipe,codigo_ano,level_2,depreciacao_mensal
0,001001-4,1992-1,1,-0.004580
1,001001-4,1992-1,2,-0.003784
2,001001-4,1992-1,3,-0.005765
3,001001-4,1992-1,4,-0.004083
4,001001-4,1992-1,5,-0.005175
...,...,...,...,...
9865276,098002-1,32000-1,9899891,-0.002096
9865277,098002-1,32000-1,9899892,-0.016264
9865278,098002-1,32000-1,9899893,0.000200
9865279,098002-1,32000-1,9899894,-0.004380


In [73]:
depreciacao_media = df_depreciacao.groupby(['codigo_fipe', 'codigo_ano'])['depreciacao_mensal'].mean().reset_index()
depreciacao_media['depreciacao_anual'] = (1 + depreciacao_media['depreciacao_mensal'])**12 - 1

depreciacao_media


,codigo_fipe,codigo_ano,depreciacao_mensal,depreciacao_anual
0,001001-4,1992-1,-0.003292,-0.038798
1,001001-4,1993-1,-0.002686,-0.031761
2,001001-4,1994-1,-0.003065,-0.036163
3,001001-4,1995-1,-0.003390,-0.039934
4,001001-4,1995-2,-0.004310,-0.050513
...,...,...,...,...
34610,097002-6,32000-1,-0.004927,-0.057553
34611,098001-3,2023-1,-0.005059,-0.059049
34612,098001-3,32000-1,-0.004978,-0.058132
34613,098002-1,2023-1,-0.005059,-0.059049


In [ ]:
depreciacao_media

,codigo_fipe,codigo_ano,depreciacao_mensal,depreciacao_anual
0,001001-4,1992-1,-0.329315,-3.880984
1,001001-4,1993-1,-0.268892,-3.179415
2,001001-4,1994-1,-0.306633,-3.618168
3,001001-4,1995-1,-0.338997,-3.992962
4,001001-4,1995-2,-0.431310,-5.054690
...,...,...,...,...
34610,097002-6,32000-1,-0.492780,-5.755699
34611,098001-3,2023-1,-0.505956,-5.905332
34612,098001-3,32000-1,-0.497874,-5.813576
34613,098002-1,2023-1,-0.505951,-5.905273


In [ ]:
depreciacao_media.sort_values(by = ["depreciacao_anual"], ascending = False)

,codigo_fipe,codigo_ano,depreciacao_mensal,depreciacao_anual
7527,004055-0,1988-1,0.233003,2.832150
10437,005027-0,1985-1,0.231930,2.818939
7529,004055-0,1990-1,0.230756,2.804486
7528,004055-0,1989-1,0.228146,2.772369
10443,005027-0,1996-1,0.220931,2.683629
...,...,...,...,...
29852,029059-9,2005-1,-1.233889,-13.842038
11639,005204-3,2005-1,-1.234282,-13.846151
11514,005184-5,2004-1,-1.237617,-13.881051
11513,005184-5,2003-1,-1.251950,-14.030908


Os modelos:

1.   Chevrolet Opala Diplomata/Diplom
2.   Volkswagen Fusca 1600
3.   Chevrolet Opala Comodoro/Comod

Apareceram em ambas as listas, mostrando ser um bom investimento.